## Take Home Assessment - Fetch

#### Problem Statement

Given a balance scale and 9 gold bars of the same size and look. You don’t know the exact weight of each bar,
but you know they all weigh the same, except for one fake bar. It weighs less than others. You need to find the fake
gold bar by only bars and balance scales.
You can only place gold bars on scale plates (bowls) and find which scale weighs more or less.

Challenge
1. Play around with the website and find the best algorithm (minimum number of weighings for any possible
fake bar position) to find the fake gold bar.
2. Create the test automation project using any preferred language to perform
a. clicks on buttons (“Weigh”, “Reset”)
b. Getting the measurement results (field between the 'bowls')
c. filling out the bowls grids with bar numbers (0 to 8)
d. getting a list of weighing
e. Clicking on the gold bar number at the bottom of the website and checking for the alert message
3. Code the algorithm from step 1 which uses a set of actions from step 2 to find the fake gold bar
The algorithm should populate and weigh gold bars until a fake one is found, click on a fake bar number, output the
alert message, number of weighing, and list of weighing made.

### Algorithm (Binary Search)

#### 1. Divide the input bar into 2 halves evenly without using the last element and weigh them. 
#### 2. If the weighing is equal the last bar is fake 
#### 3. If the weighting is not equal, go to step one with the array which has smaller weight, continue until you are left with 1 element on both side and then return the result

#### Essential we are dividing the problem into two parts and then searching in the resulting half, a pattern used in Binary Search


## Tools Used - Selenium + Chrome Driver + Python

### Importing the required Libraries  

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By ## To access elements from DOM
import time ## To add Delay
import re

### Connecting to the Test Web Page 

In [11]:
url = 'http://sdetchallenge.fetch.com/'
driver = webdriver.Chrome()
driver.get(url)

The chromedriver version (123.0.6312.122) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (124.0.6367.62); currently, chromedriver 124.0.6367.60 is recommended for chrome 124.*, so it is advised to delete the driver in PATH and retry


### Defining Functions which will be used for testing 

In [17]:
## Function to add numbers to the Grid for Weighing
def add_numbers(bar):
    bar_len=len(bar)
    index=bar_len//2   ## Dividing the given bar into two halves two add equal numbers to the left and right grid
    for i in range(index):
        val = driver.find_element(by=By.ID, value=f"left_{i}")
        val.send_keys(f"{bar[i]}")
    for i in range(index):
        val = driver.find_element(by=By.ID, value=f"right_{i}")
        val.send_keys(f"{bar[i+index]}")

## Function to get the weigh list       
def get_recent_weight():
    weigh_button = driver.find_element(by=By.ID, value="weigh")
    weigh_button.click()
    time.sleep(3)  ## The weigh doesn't show up immediately, so added some delay to it
    weight_list = driver.find_elements(by=By.TAG_NAME,value="li")
    return weight_list[-1].text
   
## Function to get two array from the weight string 
def clean_weight_string(string):
    match = re.match(r'\[(.*?)\] (\W) \[(.*?)\]', string)
    ## Extract the arrays
    left_array = list(map(int, match.group(1).split(',')))
    right_array = list(map(int, match.group(3).split(',')))

    ## Extract the operator
    operator = match.group(2)
    return left_array,right_array,operator

## Function to reset the grid
def do_reset():
    reset_button = driver.find_element(By.CSS_SELECTOR, "div > button#reset:not(.result > button#reset)")
    reset_button.click()

##### Driver Function

In [18]:
def find_fake_bar(bar):
    
    ## adding the number to the boxes
    add_numbers(bar)
    
    ## Get the weigh
    weight_string=get_recent_weight()
    
    ## Reset the two grids
    do_reset()
    
    ## Check the condition
    left_array,right_array,operator=clean_weight_string(weight_string)
  
    ## If both side has only one element, we have the answer
    if len(left_array)==1:
        if operator=='<':
            val = driver.find_element(by=By.ID, value=f"coin_{left_array[0]}")
            val.click()
        else:
            val = driver.find_element(by=By.ID, value=f"coin_{right_array[0]}")
            val.click()   
        return
      
    ## If both arrays are equal, last element is fake 
    elif operator=='=':
        val = driver.find_element(by=By.ID, value=f"coin_{len(bar)-1}")
        val.click()
        return 
    elif operator=='<':
        find_fake_bar(left_array)
    else:
        find_fake_bar(right_array)

In [20]:
bar=[0,1,2,3,4,5,6,7,8]
find_fake_bar(bar)  ## Calling the main function

### Run the below cell to show the number of weighing, and list of weighing made

In [21]:
## Find all elements with the tag name 'li'
li_elements = driver.find_elements(By.TAG_NAME, 'li')

## Get the list count
list_count = len(li_elements)

## Create a message string
message = f"Number of weighing: {list_count}\\n"
message += "List of weighing made:\\n"
for index,element in enumerate(li_elements):
    message += str(index+1)+')'+element.text+'\\n'
    

## Display the message in an alert box
driver.execute_script(f"alert('{message}');")

### Closing the driver

In [22]:
driver.quit() 